%run ../ipynb_util_tars.py

In [9]:
import os

from qdrant_client import QdrantClient, models

client = QdrantClient(
    os.getenv("QDRANT_API_URL"),
    port=os.getenv("QDRANT_API_PORT"),
    api_key=os.getenv("QDRANT_API_KEY"),
)

/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/qdrant_client/qdrant_remote.py:130: UserWarning: Api key is used with an insecure connection.
  warnings.warn("Api key is used with an insecure connection.")


In [11]:
scroll_result = client.scroll(
    collection_name="publications",
    limit=1,  # Adjust this value as needed
    scroll_filter=models.Filter(
        must=[
            models.FieldCondition(
                key="xai",
                match=models.MatchValue(value=models.AnyVariant())
            )
        ]
    )
)

for point in scroll_result.result:
    print(point)

AttributeError: module 'qdrant_client.models' has no attribute 'AnyVariant'